In [1]:
%matplotlib TkAgg

import matplotlib.pyplot as plt
import mne
import numpy as np
from scipy.signal import butter, sosfilt, freqs

In [7]:
def standardizeMatrix(matrix):
    rows, cols = matrix.shape
    
    for i in range(rows):
        mu = np.mean(matrix[i,:])
        sigma = np.std(matrix[i,:])
        matrix[i,:] = (matrix[i,:] - mu) / sigma
    return matrix


def elimPeaks(standardizedMatrix, sigmaThreshold, sampleHalfWidthElim, sampleMinGap):
    peakList = []
    nChannel, nSample = standardizedMatrix.shape
    
    for i in range(nChannel):
        for j in range(nSample):
            if np.abs(standardizedMatrix[i,j]) > sigmaThreshold:
                peakList.append(j)
    peakList = list(set(peakList))
    peakList.sort()

    intervalList = [(thisPeak - sampleHalfWidthElim, thisPeak + sampleHalfWidthElim) for thisPeak in peakList]

    mergedIntervalList = [intervalList[0]]
    for i in range(1, len(intervalList)):
        currentInterval = mergedIntervalList[-1]
        nextInterval = intervalList[i]
        if currentInterval[1] + sampleMinGap >= nextInterval[0]:
            mergedIntervalList[-1] = (currentInterval[0], nextInterval[1])
        else:
            mergedIntervalList.append(nextInterval)
        
    startIndex = 0
    matrixList = []
    for i in range(0, len(mergedIntervalList)):
        intervalStart, intervalEnd = mergedIntervalList[i]
        thisMatrix = np.copy(data[:, startIndex:intervalStart])
        matrixList.append(thisMatrix)
        startIndex = intervalEnd + 1

    return matrixList

In [3]:
filename = 'SVD001.edf'
raw = mne.io.read_raw_edf('G:/My Drive/NeuroResearch/Data/eegCompress/ExampleLTMFiles/' + filename)

Extracting EDF parameters from G:\My Drive\NeuroResearch\Data\eegCompress\ExampleLTMFiles\SVD001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jetti\AppData\Local\Temp\ipykernel_14356\626497673.py:2: RuntimeWarning: Omitted 9 annotation(s) that were outside data range.
  raw = mne.io.read_raw_edf('G:/My Drive/NeuroResearch/Data/eegCompress/ExampleLTMFiles/' + filename)


In [4]:
print(raw)
print(raw.info)

<RawEDF | SVD001.edf, 46 x 1276416 (4986.0 s), ~39 KiB, data not loaded>
<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, F7, T7, P7, O1, F3, C3, P3, A1, Fz, Cz, Fp2, F8, T8, P8, ...
 chs: 46 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2001-01-01 04:46:55 UTC
 nchan: 46
 projs: []
 sfreq: 256.0 Hz
 subject_info: <subject_info | his_id: SVD001>
>


In [5]:
goodChannels = ['Fp1','F3','C3','P3','O1','Fp2','F4','C4','P4','O2','F7','T7','P7','F8','T8','P8','Fz','Cz','Pz']

In [29]:
lowPass = 40
sFreqNew = lowPass * 2
sigmaThreshold = 8
sampleHalfWidthElim = int(sFreqNew/2)
sampleMinGap = sFreqNew

chanList = mne.pick_channels(raw.info['ch_names'], goodChannels)

data = raw.get_data()[chanList]
nChannel, nSample = data.shape

dataProcessed = standardizeMatrix(raw.copy().resample(sFreqNew).get_data()[chanList])
dataList = elimPeaks(dataProcessed, sigmaThreshold, sampleHalfWidthElim, sampleMinGap)

In [34]:
plt.figure()
plt.plot(dataProcessed[0,:])

In [35]:
catData = np.concatenate(dataList,1)
plt.figure()
plt.plot(catData[0,:])

In [ ]:
np.savez_compressed('G:/My Drive/NeuroResearch/Data/eegCompress/processedData/' + 'processed' + filename[0:-4], dataProcessed)

# Scratch

In [ ]:
np.save('G:/My Drive/NeuroResearch/Data/eegCompress/processedData/' + filename[0:-4], data)

In [60]:
sos = butter(30, [1,40], 'bandpass', fs=256, output='sos')

for i in range(nChannel - 1):
    data[i,:] = data[i,:] - data[i + 1,:]
    data[i,:] = sosfilt(sos, data[i,:])
    data[i,:] = (data[i,:] - np.mean(data[i,:])) / np.std(data[i,:])
    #data[i,:] = data[i,:] + np.min(data[i,:])

data = data[0:nChannel - 1,:]
print(data.shape)

(18, 1276416)
